# Parte III: K-Means

In [ ]:
import os

sc.setCheckpointDir('/user/cloudera/checkpoints')

Leemos los datos de rating

In [ ]:
ratings_raw_data = sc.textFile('/user/cloudera/ratings.dat')
ratings_data = ratings_raw_data.map(lambda line: line.split(";")).map(lambda t: (t[1],t[0]))

Leemos los datos de las películas

In [ ]:
movies_raw_data = sc.textFile('/user/cloudera/movies.dat')
movies_data = movies_raw_data.map(lambda line: line.split(";")).map(lambda t: (t[0] , t[2].split('|')))

Obtenemos los conteos de (user, genero) y los vector de generos de peliculas

In [ ]:
genreUser_data = ratings_data.join(movies_data).map(lambda x:x[1]).flatMapValues(lambda x: x)
genreUser_count = genreUser_data.map(lambda x: (x,1)).reduceByKey(lambda a,b:a+b)

genres = genreUser_data.map(lambda x:x[1]).distinct().collect()

Obtenemos los vectores descriptores (BoW de generos por usuario)

In [ ]:
import numpy as np
def create_bow_genres(vec, genres):
    bow = np.zeros(len(genres))
    for v in vec:
        bow[genres.index(v[1])] = np.float(v[2])
    return bow

Entrenamos k-means

In [ ]:
userVec = genreUser_count.map(lambda x: (x[0][0], x[0][1], x[1])).groupBy(lambda x: x[0]).map(lambda x: (x[0], create_bow_genres(x[1], genres))) 
onlyBow = userVec.map(lambda x: x[1]/np.sum(x[1])).cache()

Encontramos los clusters

In [ ]:
from pyspark.mllib.clustering import KMeans
model = KMeans.train(onlyBow, k=10, maxIterations = 100)

Visualizamos los clusters con la importancia de cada genero (5 generos mas importantes por cluster)

In [ ]:
centers = model.clusterCenters

for i,c in enumerate(centers):
    perm = sorted(enumerate(c), key = lambda x:x[1], reverse=True)
    print '*** Cluster ' + str(i) + ' ***'
    for p in perm[:5]:
        print genres[p[0]] + ': ' + str(p[1])
    print ''